In [1]:
import backtrader as bt
import datetime
import pandas as pd
import math

In [2]:
from lutils.stock import LTdxHq

In [3]:
ltdxhq = LTdxHq()

In [4]:
# ll = ltdxhq.stock_list()

In [5]:
# ll[(ll['pre_close'] > 30) & (ll['pre_close'] < 50)]

In [6]:
# 399300
code = '601857' # 000032 300142 603636 600519 688567 601857
# df = ltdxhq.get_k_data_daily('300142') # 000032 300142 603636 600519
# df = ltdxhq.get_k_data_15min(code) #' # 000032 300142 603636 600519
# df = ltdxhq.to_qfq(code, df)

In [7]:
# df.index = df.index.unique(level=1)

In [8]:
df = ltdxhq.get_k_data_daily(code, qfq=True) # 000032 300142 603636 600519 601857
# df1d = ltdxhq.to_qfq(code, df1d)

In [9]:
# df.index = df.index.unique(level=1)

In [10]:
ltdxhq.close()

In [11]:
df.index = pd.to_datetime(df.index)

In [12]:
# df = df['2020-01-01':]

In [13]:
class KeltnerChannel(bt.Indicator):
    lines = ('mid', 'upper', 'lower')
    params = dict(
                ema=20,
                atr=3
                )

    plotinfo = dict(subplot=False)  # plot along with data
    plotlines = dict(
        mid=dict(ls='--'),  # dashed line
        upper=dict(_samecolor=True),  # use same color as prev line (mid)
        lower=dict(_samecolor=True),  # use same color as prev line (upper)
    )

    def __init__(self):
        self.l.mid = bt.ind.EMA(period=self.p.ema)
        self.l.upper = self.l.mid + bt.ind.ATR(period=self.p.ema) * self.p.atr
        self.l.lower = self.l.mid - bt.ind.ATR(period=self.p.ema) * self.p.atr

In [14]:
class KeltnerChannelStrategy(bt.Strategy):

    def __init__(self):
        self.keltner = KeltnerChannel()
    
    def log(self, txt):
        ''' Logging function for this strategy'''
        dt = self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status == order.Completed:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Size: %d, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (
                        order.executed.size,
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm,
                    )
                )

            else:  # Sell
                self.log(
                    'SELL EXECUTED, Size: %d, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (
                        order.executed.size,
                        order.executed.price,
                        order.executed.value,
                        order.executed.comm
                    )
                )

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
        
    def next(self):
        if self.keltner.l.lower[0] > self.data[0]:
            price = self.data0.close[0]
            cash = self.broker.get_cash()
            share = int(math.floor((cash)/price))
            
            self.buy(size=share)
        elif self.keltner.l.upper[0] < self.data[0]:
            
            self.order = self.close()
#             self.sell()

In [15]:
cerebro = bt.Cerebro()

# Add Benchmark
# benchmark = get_security_data(BENCHMARK_TICKER, START, END)
benchdata = bt.feeds.PandasData(dataname=df, name='SPY', plot=True)
cerebro.adddata(benchdata)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Add Strategy
cerebro.addstrategy(KeltnerChannelStrategy)

cerebro.broker.setcash(10000.0)

cerebro.broker.setcommission(0.0005)

results = cerebro.run(stdstats=False)

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2007-12-04T00:00:00, Order Canceled/Margin/Rejected
2007-12-05T00:00:00, Order Canceled/Margin/Rejected
2008-01-23T00:00:00, BUY EXECUTED, Size: 552, Price: 18.07, Cost: 9975.27, Comm 4.99
2008-01-24T00:00:00, BUY EXECUTED, Size: 1, Price: 18.35, Cost: 18.35, Comm 0.01
2009-02-11T00:00:00, SELL EXECUTED, Size: -553, Price: 8.27, Cost: 9993.62, Comm 2.29
2009-08-18T00:00:00, BUY EXECUTED, Size: 488, Price: 9.30, Cost: 4538.09, Comm 2.27
2010-04-28T00:00:00, BUY EXECUTED, Size: 3, Price: 8.59, Cost: 25.78, Comm 0.01
2010-10-12T00:00:00, SELL EXECUTED, Size: -491, Price: 8.00, Cost: 4563.86, Comm 1.96
2011-05-25T00:00:00, Order Canceled/Margin/Rejected
2011-08-09T00:00:00, BUY EXECUTED, Size: 528, Price: 7.38, Cost: 3896.77, Comm 1.95
2011-08-10T00:00:00, BUY EXECUTED, Size: 4, Price: 7.42, Cost: 29.67, Comm 0.01
2014-07-29T00:00:00, SELL EXECUTED, Size: -532, Price: 6.61, Cost: 3926.44, Comm 1.76
2015-08-26T00:00:00, Order Canceled/Margin/Rejected
2016-

In [16]:
cerebro.plot(
    iplot=False,
    start=datetime.date(2020, 1, 1),
    end=datetime.date(2021, 9, 30),
    style='candlestick',
    barup='red',
    bardown='green',
)

[[<Figure size 1707x960 with 2 Axes>]]